In [1]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()

from typing import List, Dict, Callable
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

DEPLOYMENT_NAME="gpt-35-turbo"

llm = AzureChatOpenAI(
    openai_api_base=RESOURCE_ENDPOINT,
    openai_api_version="2023-03-15-preview",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type = "azure",
    temperature=1.0
)

In [2]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: AzureChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()
        
    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")

In [3]:
class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function
        
    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

In [4]:
character_names_and_roles = ["Edgin Darvis, homem humano bardo, líder do grupo. Sempre tentando se dar bem e ganhar dinheiro", 
                             "Holga Kilgore, mulher humana barbara, cabeça quente e sempre pronta para luta", 
                             "Simon Aumar, homem elfo feiticeiro, tem baixa auto-estima, mas sempre resolve quando sob pressão", 
                             "Doric, mulher thiefiling druida, pode se transformar em uma CorujaUrso gigante"]
character_names = []

for character in character_names_and_roles:
    name = character.split(',')[0].strip()
    character_names.append(name)
    
    
storyteller_name = "Dungeon Master"
quest = "Encontrem o elmo da disjunção, escondido nas cavernas de Underdark"
word_limit = 50 # word limit for task brainstorming

In [5]:
game_description = f"""Aqui está o tópico para um jogo de Dungeons & Dragons: {quest}.
        Os personagens são: {*character_names,}.
        A história é narrada por: {storyteller_name}."""

player_descriptor_system_message = SystemMessage(
    content="Você consegue adicionar detalhes à descrição de um personagem de Dungeons & Dragons.")

def generate_character_description(character_name):
    character_specifier_prompt = [
        player_descriptor_system_message,
        HumanMessage(content=
            f"""{game_description}
            Por favor responda com uma descrição criativa do personagem, {character_name}, em {word_limit} palavras ou menos. 
            Fale direto para o {character_name}.
            Não adicione nada a mais."""
            )
    ]
    character_description = llm(character_specifier_prompt).content
    return character_description

def generate_character_system_message(character_name, character_description):
    return SystemMessage(content=(
    f"""{game_description}
    Seu nome é {character_name}. 
    A descrição do seu personagem é: {character_description}.
    Você irá propor ações que irá fazer e o {storyteller_name} irá explicar o resultado destas ações.
    Fale em primeira pessoa na perspectiva do {character_name}.
    Para descrever seus movimentos, envolva sua descrição em '*'.
    Não mude de papel!
    Não fale na perspectiva de outra pessoa.
    Lembre-se, você é {character_name}.
    Pare de falar no momento que terminar a fala na sua perspectiva.
    Nunca esqueça de limitar sua resposta a {word_limit} palavras!
    Não adicione nada a mais.
    """
    ))

character_descriptions = [generate_character_description(character_name) for character_name in character_names_and_roles]
character_system_messages = [generate_character_system_message(character_name, character_description) for character_name, character_description in zip(character_names_and_roles, character_descriptions)]

storyteller_specifier_prompt = [
    player_descriptor_system_message,
    HumanMessage(content=
        f"""{game_description}
        Por favor responda com uma descrição criativa do narrador, {storyteller_name}, em {word_limit} palavras ou menos. 
        Fale direto para o {storyteller_name}.
        Não adicione nada a mais."""
        )
]
storyteller_description = llm(storyteller_specifier_prompt).content

storyteller_system_message = SystemMessage(content=(
f"""{game_description}
Você é o narrador, {storyteller_name}. 
Sua descrição é a seguinte: {storyteller_description}.
Os outros jogadores irão propor ações a serem tomadas e você irá explicar o que acontece quando eles tomam essas ações.
Fale em primeira pessoa da perspectiva do {storyteller_name}.
Não mude de papel!
Não fale na perspectiva de outra pessoa..
embre-se, você é o narrador, {storyteller_name}.
Pare de falar no momento que terminar a fala na sua perspectiva.
Nunca esqueça de limitar sua resposta a {word_limit} palavras!
Não adicione nada a mais.
"""
))

In [6]:
print('Storyteller Description:')
print(storyteller_description)
for character_name, character_description in zip(character_names, character_descriptions):
    print(f'{character_name} Description:')
    print(character_description)

Storyteller Description:
Narrador, você é um mestre hábil das histórias e dos mistérios. Sua voz profunda e cadenciada comanda a atenção e o respeito de todos. Cada palavra que sai de sua boca é carregada de um peso e uma consequência que farão todo o grupo prestar atenção.
Edgin Darvis Description:
Edgin Darvis é um bardo humano esperto e astuto. Ele lidera o grupo com um olhar ambicioso, sempre pensando em como maximizar seus lucros. Com sua habilidade musical excepcional, Edgin é capaz de persuadir até mesmo os mais obstinados com sua voz encantadora. Mas não subestime sua capacidade de luta. Quando se trata de proteger seus interesses, Edgin não tem medo de lutar sujo.
Holga Kilgore Description:
Holga Kilgore é uma selvagem sem medo. Com seus cabelos louros amarrados em um coque bagunçado e um olhar ardente em seus olhos, ela é uma força a ser reconhecida. Vestida em couro resistente e armada com uma machadinha, ela está sempre pronta para correr para a batalha.
Simon Aumar Descrip

In [10]:
quest_specifier_prompt = [
    SystemMessage(content="Você pode tornar uma tarefa mais específica."),
    HumanMessage(content=
        f"""{game_description}
        
        Você é o narrador, {storyteller_name}.
        Por favor torne uma aventura mais específica. Seja criativo e imaginativo.
        Por favor, responda com a aventura em {word_limit} palavras ou menos. 
        Fale diretamente para os personagens: {*character_names_and_roles,}.
        Não adicione nada a mais."""
        )
]
specified_quest = llm(quest_specifier_prompt).content

print(f"Original quest:\n{quest}\n")
print(f"Detailed quest:\n{specified_quest}\n")

Original quest:
Encontrem o elmo da disjunção, escondido nas cavernas de Underdark

Detailed quest:
Edgin Darvis, Holga Kilgore, Simon Aumar e Doric, vocês estão no fundo das cavernas de Underdark, indo em direção à sala do trono do dragão vermelho, Amalthia. Rumores dizem que o elmo da disjunção que vocês procuram está em sua posse. Cuidado com as armadilhas e lacaios pelo caminho. Boa sorte.



In [11]:
#Main Loop

llm2 = AzureChatOpenAI(
    openai_api_base=RESOURCE_ENDPOINT,
    openai_api_version="2023-03-15-preview",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type = "azure",
    temperature=0.2
)

characters = []
for character_name, character_system_message in zip(character_names, character_system_messages):
    characters.append(DialogueAgent(
        name=character_name,
        system_message=character_system_message, 
        model=llm))
storyteller = DialogueAgent(name=storyteller_name,
                     system_message=storyteller_system_message, 
                     model=llm2)

In [12]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    """
    If the step is even, then select the storyteller
    Otherwise, select the other characters in a round-robin fashion.
    
    For example, with three characters with indices: 1 2 3
    The storyteller is index 0.
    Then the selected index will be as follows:

    step: 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16

    idx:  0  1  0  2  0  3  0  1  0  2  0  3  0  1  0  2  0
    """
    if step % 2 == 0:
        idx = 0
    else:
        idx = (step//2) % (len(agents)-1) + 1
    return idx

In [13]:
max_iters = 10
n = 0

simulator = DialogueSimulator(
    agents=[storyteller] + characters,
    selection_function=select_next_speaker
)
simulator.reset()
simulator.inject(storyteller_name, specified_quest)
print(f"({storyteller_name}): {specified_quest}")
print('\n')

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print('\n')
    n += 1

(Dungeon Master): Edgin Darvis, Holga Kilgore, Simon Aumar e Doric, vocês estão no fundo das cavernas de Underdark, indo em direção à sala do trono do dragão vermelho, Amalthia. Rumores dizem que o elmo da disjunção que vocês procuram está em sua posse. Cuidado com as armadilhas e lacaios pelo caminho. Boa sorte.


(Edgin Darvis): Vamos andar com cuidado para não cairmos em nenhuma armadilha. Mas não podemos demorar muito, precisamos encontrar logo o elmo da disjunção e conseguir um bom preço por ele. Alguém tem alguma ideia de por onde começar a procurar?


(Dungeon Master): Vocês avançam cautelosamente pelas cavernas, evitando armadilhas e enfrentando os lacaios de Amalthia. Enquanto exploram, vocês encontram um mapa antigo que mostra a localização da sala do trono. Mas cuidado, o mapa também indica a presença de um monstro guardião. O que vocês fazem?


(Holga Kilgore): *Eu pego o mapa e mostro aos meus companheiros.* "Vamos ter que enfrentar esse monstro guardião, mas não podemos d

In [14]:
max_iters = 20


while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print('\n')
    n += 1

(Holga Kilgore): *Nós vamos aceitar o desafio, mas não podemos permitir que os goblins saibam que estamos atrás do colar. Vamos nos infiltrar furtivamente e tentar pegá-lo sem sermos vistos. Depois, podemos voltar e pegar o elmo com Amalthia."*


(Dungeon Master): Vocês se infiltram furtivamente na caverna dos goblins e encontram o colar de rubis. Mas, ao tentar sair, vocês são descobertos e precisam lutar contra os goblins. Com suas habilidades de luta, vocês conseguem derrotá-los e pegar o colar. Agora, vocês podem voltar para Amalthia e trocar o colar pelo elmo da disjunção. Mas cuidado, o dragão pode ter outras surpresas guardadas para vocês.


(Simon Aumar): *Eu proponho que usemos a nossa astúcia para fugir da sala do trono com o elmo sem chamar a atenção do dragão. Podemos usar um truque de ilusão ou um feitiço de invisibilidade para sairmos ilesos.*
Dungeon Master: Você usa suas habilidades de feitiçaria para lançar um feitiço de invisibilidade sobre si mesmo e seus companheiro

KeyError: 'content'